In [2]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Daftar sparepart dan mapping ke indeks
spareparts = ['spion', 'busi', 'lampu_depan', 'lampu_belakang', 'sein']
sparepart_to_index = {sp: i for i, sp in enumerate(spareparts)}

def derive_bbox(sparepart, width=150, height=150):
    if sparepart == 'spion':
        xmin = int(0.3*width)
        ymin = int(0.4*height)
        xmax = int(0.7*width)
        ymax = int(0.9*height)
    elif sparepart == 'busi':
        xmin = int(0.4*width)
        ymin = int(0.1*height)
        xmax = int(0.6*width)
        ymax = int(0.8*height)
    elif sparepart == 'lampu_depan':
        xmin = int(0.3*width)
        ymin = int(0.3*height)
        xmax = int(0.7*width)
        ymax = int(0.7*height)
    elif sparepart == 'lampu_belakang':
        xmin = int(0.35*width)
        ymin = int(0.4*height)
        xmax = int(0.65*width)
        ymax = int(0.7*height)
    elif sparepart == 'sein':
        xmin = int(0.3*width)
        ymin = int(0.2*height)
        xmax = int(0.7*width)
        ymax = int(0.8*height)
    return [xmin, ymin, xmax, ymax]

class SparepartDataset(Dataset):
    def __init__(self, root_dir='data/train', transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        # Load all images
        for sp in spareparts:
            for q in ['baik','sedang','buruk']:
                d = os.path.join(root_dir, sp, q)
                files = os.listdir(d)
                for f in files:
                    if f.endswith('.png'):
                        img_path = os.path.join(d, f)
                        self.samples.append((img_path, sp))
                        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, sp = self.samples[idx]
        img = Image.open(img_path).convert('RGB')
        label = sparepart_to_index[sp]
        bbox = derive_bbox(sp)
        w, h = img.size
        bbox_norm = [bbox[0]/w, bbox[1]/h, bbox[2]/w, bbox[3]/h]
        
        if self.transform:
            img = self.transform(img)
        
        return img, label, torch.tensor(bbox_norm, dtype=torch.float32)

# Simple CNN Model with two heads: classification & bbox regression
class SparepartModel(nn.Module):
    def __init__(self, num_classes=5):
        super(SparepartModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,32,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,1,1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(128*(224//8)*(224//8),256),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.classifier = nn.Linear(256, num_classes)
        self.bbox_regressor = nn.Linear(256,4)
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        class_logits = self.classifier(x)
        bbox = self.bbox_regressor(x)
        return class_logits, bbox

In [2]:
import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image, UnidentifiedImageError
import logging
from torchvision import models
import h5py  # Import h5py for HDF5 support

# ============================
# 1. Setup and Configuration
# ============================

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device: {device}')

# ============================
# 2. Data Preparation
# ============================

# Daftar sparepart dan mapping ke indeks
spareparts = [
    'spion', 'knalpot', 'spion_rusak', 'motor_lecet',
    'honda_beat_biru_putih', 'honda_beat_hijau', 'honda_beat_hitam', 'honda_beat_silver',
    'honda_vario_hitam', 'honda_vario_putih',
    'yamaha_aerox_hitam', 'yamaha_aerox_kuning', 'yamaha_aerox_putih',
    'yamaha_nmax_hitam', 'yamaha_nmax_merah', 'yamaha_nmax_putih',
    'plat_nomor'
]
sparepart_to_index = {sp: i for i, sp in enumerate(spareparts)}
num_classes = len(spareparts)

def derive_bbox(sparepart, width=224, height=224):
    """
    Menghasilkan bounding box yang dinormalisasi berdasarkan nama sparepart.
    Menyesuaikan dengan fungsi generate_dummy_image.
    """
    if sparepart in ['spion', 'spion_rusak']:
        xmin = int(0.3 * width)
        ymin = int(0.4 * height)
        xmax = int(0.7 * width)
        ymax = int(0.9 * height)
    elif sparepart == 'knalpot':
        xmin = int(0.3 * width)
        ymin = int(0.3 * height)
        xmax = int(0.7 * width)
        ymax = int(0.7 * height)
    elif sparepart == 'motor_lecet':
        xmin = int(0.2 * width)
        ymin = int(0.2 * height)
        xmax = int(0.8 * width)
        ymax = int(0.8 * height)
    elif sparepart.startswith('honda') or sparepart.startswith('yamaha'):
        xmin = int(0.2 * width)
        ymin = int(0.3 * height)
        xmax = int(0.8 * width)
        ymax = int(0.6 * height)
    elif sparepart == 'plat_nomor':
        xmin = int(0.1 * width)
        ymin = int(0.4 * height)
        xmax = int(0.9 * width)
        ymax = int(0.6 * height)
    else:
        # Default bounding box jika sparepart tidak dikenali
        xmin = int(0.3 * width)
        ymin = int(0.3 * height)
        xmax = int(0.7 * width)
        ymax = int(0.7 * height)
    return [xmin, ymin, xmax, ymax]

class SparepartDataset(Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, sp = self.samples[idx]
        try:
            img = Image.open(img_path).convert('RGB')
        except (UnidentifiedImageError, IOError) as e:
            logging.error(f"Error loading image {img_path}: {e}")
            img = Image.new('RGB', (224, 224), (0, 0, 0))
        
        label = sparepart_to_index.get(sp)
        if label is None:
            logging.error(f"Label untuk sparepart '{sp}' tidak ditemukan di {img_path}.")
            raise ValueError(f"Label tidak valid untuk gambar {img_path}")
        
        if self.transform:
            img = self.transform(img)
        
        bbox = derive_bbox(sp, width=224, height=224)
        bbox_norm = [bbox[0]/224, bbox[1]/224, bbox[2]/224, bbox[3]/224]
        
        return img, label, torch.tensor(bbox_norm, dtype=torch.float32)

def load_dataset(root_dir):
    samples = []
    for sp in spareparts:
        d = os.path.join(root_dir, sp)
        if not os.path.isdir(d):
            logging.warning(f"Directory {d} tidak ada. Melewati.")
            continue
        files = os.listdir(d)
        for f in files:
            if f.endswith('.png'):
                img_path = os.path.join(d, f)
                samples.append((img_path, sp))
    logging.info(f"Loaded {len(samples)} samples dari {root_dir}")
    return samples

def validate_dataset(samples):
    valid_samples = []
    invalid_files = []
    for img_path, sp in samples:
        if sp not in sparepart_to_index:
            logging.error(f"Sparepart '{sp}' tidak dikenali. Melewati {img_path}.")
            continue
        try:
            img = Image.open(img_path).convert('RGB')
            img.close()
            valid_samples.append((img_path, sp))
        except (UnidentifiedImageError, IOError) as e:
            invalid_files.append(img_path)
            logging.error(f"Gambar tidak valid {img_path}: {e}")
    
    if invalid_files:
        logging.warning(f"Total gambar tidak valid yang ditemukan: {len(invalid_files)}")
        logging.info(f"Menghapus {len(invalid_files)} gambar tidak valid.")
    else:
        logging.info("Semua gambar valid.")
    
    return valid_samples

# Transformasi untuk data dengan augmentasi
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# ============================
# 3. Model Definition
# ============================

# Menggunakan pre-trained ResNet50 dengan modifikasi untuk multi-task learning
class SparepartModel(nn.Module):
    def __init__(self, num_classes=17, pretrained=True):
        super(SparepartModel, self).__init__()
        self.backbone = models.resnet50(pretrained=pretrained)
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()
        
        # Head untuk klasifikasi
        self.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
        # Head untuk bounding box regression
        self.bbox_regressor = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 4),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        features = self.backbone(x)
        class_logits = self.classifier(features)
        bbox = self.bbox_regressor(features)
        return class_logits, bbox

# ============================
# 4. Helper Functions for .h5
# ============================

def save_model_h5(model, filepath):
    """
    Saves the model's state_dict to an HDF5 (.h5) file.

    Args:
        model (nn.Module): The PyTorch model to save.
        filepath (str): The path where the .h5 file will be saved.
    """
    state_dict = model.state_dict()
    with h5py.File(filepath, 'w') as f:
        for key, value in state_dict.items():
            f.create_dataset(key, data=value.cpu().numpy())
    logging.info(f"Model saved to {filepath} in .h5 format.")

def load_model_h5(model, filepath):
    """
    Loads the model's state_dict from an HDF5 (.h5) file.

    Args:
        model (nn.Module): The PyTorch model to load the state_dict into.
        filepath (str): The path to the .h5 file.
    """
    with h5py.File(filepath, 'r') as f:
        state_dict = {}
        for key in f.keys():
            state_dict[key] = torch.tensor(f[key][...])
    model.load_state_dict(state_dict)
    logging.info(f"Model loaded from {filepath}.")

# ============================
# 5. Data Loading
# ============================

# Load dan validasi data training
train_samples = load_dataset('data/train')
train_samples = validate_dataset(train_samples)

# Pastikan tidak ada sampel dengan label tidak valid
train_samples = [s for s in train_samples if s[1] in sparepart_to_index]

train_dataset = SparepartDataset(train_samples, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, pin_memory=True)

# Load dan validasi data validasi
validation_samples = load_dataset('data/validation')
validation_samples = validate_dataset(validation_samples)
validation_samples = [s for s in validation_samples if s[1] in sparepart_to_index]
validation_dataset = SparepartDataset(validation_samples, transform=val_transform)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=True)

# ============================
# 6. Model Initialization
# ============================

# Inisialisasi model
model = SparepartModel(num_classes=num_classes, pretrained=True)
model.to(device)
logging.info("Model initialized and moved to device.")

# ============================
# 7. Loss Function and Optimizer
# ============================

# Menghitung class weights
class_counts = {}
for _, sp in train_samples:
    class_counts[sp] = class_counts.get(sp, 0) + 1
epsilon = 1e-6
class_weights = []
for sp in spareparts:
    count = class_counts.get(sp, 0)
    if count > 0:
        class_weights.append(1.0 / (count + epsilon))
    else:
        class_weights.append(1.0)  # Atur bobot default untuk kelas tanpa sampel
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

criterion_class = nn.CrossEntropyLoss(weight=class_weights)
criterion_bbox = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

logging.info("Loss functions, optimizer, and scheduler are set up.")

# ============================
# 8. Training Loop
# ============================

# Training loop dengan perbaikan
num_epochs = 20
best_val_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels, bboxes in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        bboxes = bboxes.to(device)
        
        optimizer.zero_grad()
        
        outputs_class, outputs_bbox = model(images)
        loss_class = criterion_class(outputs_class, labels)
        loss_bbox = criterion_bbox(outputs_bbox, bboxes)
        loss = loss_class + loss_bbox
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    logging.info(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    
    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels, bboxes in validation_loader:
            images = images.to(device)
            labels = labels.to(device)
            bboxes = bboxes.to(device)
            
            outputs_class, outputs_bbox = model(images)
            loss_class = criterion_class(outputs_class, labels)
            loss_bbox = criterion_bbox(outputs_bbox, bboxes)
            loss = loss_class + loss_bbox
            val_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(outputs_class, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss = val_loss / len(validation_dataset)
    val_accuracy = 100 * correct / total
    logging.info(f'Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')
    
    # Update learning rate
    scheduler.step()
    
    # Menyimpan model terbaik dalam format .h5
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        save_model_h5(model, 'best_model_sparepart.h5')
        logging.info(f"Model terbaik disimpan dengan akurasi {best_val_accuracy:.2f}% dalam format .h5")

logging.info("Training selesai.")

# Menyimpan model akhir dalam format .h5
save_model_h5(model, 'model_sparepart_final.h5')
logging.info("Model disimpan sebagai 'model_sparepart_final.h5'")

# ============================
# 9. Optional: Loading the Model
# ============================

# Jika Anda ingin memuat model dari file .h5, gunakan kode berikut:
model = SparepartModel(num_classes=num_classes, pretrained=False)  # Set pretrained=False if loading state_dict
load_model_h5(model, 'best_model_sparepart.h5')
model.to(device)
model.eval()

INFO: Using device: cpu
INFO: Loaded 212 samples dari data/train
ERROR: Gambar tidak valid data/train/spion/spion_57.png: cannot identify image file '/Users/Adam/Desktop/bangkit/percobaan capstone/data/train/spion/spion_57.png'
INFO: Menghapus 1 gambar tidak valid.
INFO: Loaded 54 samples dari data/validation
INFO: Semua gambar valid.
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
INFO: Model initialized and move

SparepartModel(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
  

In [1]:
pip install h5py

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
import logging
import sys

# ------------------------- Setup Logging -------------------------
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)

# ------------------------- Define Spareparts List -------------------------
spareparts = [
    'spion', 'knalpot', 'spion_rusak', 'motor_lecet',
    'honda_beat_biru_putih', 'honda_beat_hijau', 'honda_beat_hitam', 'honda_beat_silver',
    'honda_vario_hitam', 'honda_vario_putih',
    'yamaha_aerox_hitam', 'yamaha_aerox_kuning', 'yamaha_aerox_putih',
    'yamaha_nmax_hitam', 'yamaha_nmax_merah', 'yamaha_nmax_putih',
    'plat_nomor'
]

# ------------------------- Define the Model Class -------------------------
class SparepartModel(nn.Module):
    def __init__(self, num_classes=17):
        super(SparepartModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 112x112
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 56x56
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(2)   # 28x28
        )
        self.fc = nn.Sequential(
            nn.Linear(128 * 28 * 28, 256),
            nn.ReLU(),
            nn.Dropout(0.5)
        )
        self.classifier = nn.Linear(256, num_classes)
        self.bbox_regressor = nn.Linear(256, 4)
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        class_logits = self.classifier(x)
        bbox = self.bbox_regressor(x)
        return class_logits, bbox

# ------------------------- Device Configuration -------------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f"Using device: {device}")

# ------------------------- Initialize and Load the Model -------------------------
model = SparepartModel(num_classes=len(spareparts))
model_path = 'model_sparepart.pth'  # Update the path if necessary

try:
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    logging.info(f"Model loaded successfully from '{model_path}'.")
except FileNotFoundError:
    logging.error(f"Model file '{model_path}' not found. Please ensure the file exists.")
    sys.exit(1)
except Exception as e:
    logging.error(f"Error loading the model: {e}")
    sys.exit(1)

# ------------------------- Define Image Transform -------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# ------------------------- Open the Camera -------------------------
cap = cv2.VideoCapture(0)  # 0 is the default camera

if not cap.isOpened():
    logging.error("Tidak dapat membuka kamera. Pastikan kamera tersedia.")
    sys.exit(1)

logging.info("Arahkan kamera ke objek (sparepart).")
logging.info("Tekan 'Space' untuk mengambil gambar, 'q' untuk keluar tanpa mengambil gambar.")

captured = False
frame = None  # Initialize frame variable

while True:
    ret, frame = cap.read()
    if not ret:
        logging.error("Gagal menangkap frame dari kamera.")
        break

    cv2.imshow("Camera (Tekan 'Space' untuk memfoto)", frame)
    key = cv2.waitKey(1) & 0xFF
    # Uncomment the line below for debugging key presses
    # logging.debug(f"Key pressed: {key}")

    if key == ord('q'):
        # Keluar tanpa mengambil gambar
        logging.info("Keluar tanpa mengambil gambar.")
        break
    elif key == ord(' '):  # Space key
        # User menekan space, ambil foto ini untuk prediksi
        captured = True
        logging.info("Gambar diambil untuk prediksi.")
        break

# ------------------------- If Image Captured, Perform Prediction -------------------------
if captured and frame is not None:
    try:
        # Preprocess the captured frame
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(img_rgb)

        input_tensor = transform(pil_img).unsqueeze(0).to(device)  # (1,3,224,224)
        with torch.no_grad():
            class_logits, pred_bbox = model(input_tensor)
            _, preds = torch.max(class_logits, 1)

        # Denormalisasi bounding box
        pred_box = pred_bbox[0].cpu().numpy()
        h, w = 224, 224  # Since we resized the image to 224x224
        xmin = int(pred_box[0] * w)
        ymin = int(pred_box[1] * h)
        xmax = int(pred_box[2] * w)
        ymax = int(pred_box[3] * h)

        # Validasi prediksi kelas
        try:
            pred_class = spareparts[preds[0].item()]
        except IndexError:
            logging.error(f"Predicted class index {preds[0].item()} out of range.")
            pred_class = "Unknown"

        logging.info(f"Predicted Class: {pred_class}")
        logging.info(f"Predicted Bounding Box: ({xmin}, {ymin}), ({xmax}, {ymax})")

        # Menampilkan prediksi pada gambar hasil foto
        # Resize frame ke (224,224) agar sesuai dengan bbox yang diprediksi
        resized_frame = cv2.resize(frame, (224, 224))
        resized_frame = np.ascontiguousarray(resized_frame)

        # Gambar bounding box
        cv2.rectangle(resized_frame, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
        # Tulis nama kelas
        cv2.putText(resized_frame, pred_class, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (255, 0, 0), 1)

        # Tampilkan hasil prediksi
        cv2.imshow("Hasil Prediksi", resized_frame)
        logging.info("Tekan sembarang tombol pada jendela gambar untuk menutup.")
        cv2.waitKey(0)
        cv2.destroyWindow("Hasil Prediksi")

    except Exception as e:
        logging.error(f"Terjadi kesalahan saat memproses gambar: {e}")

# ------------------------- Release Resources -------------------------
cap.release()
cv2.destroyAllWindows()
logging.info("Selesai.")

INFO: Using device: cpu
/var/folders/z6/9n120bs12z1_j_qxhrdxr2300000gp/T/ipykernel_7767/90200859.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(t